# Temporal Graph Data Generation for Single-Camera MOT
In this notebook, we are going to generate the temporal graph dataset for the MOT task. Based on the previously preprocessed data the required manipulations are performed. For the sake of simplicity, a single camera scenario is considered. 

In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
# pre-set the number of frames we want to look at 
n_frames = 10

### 1. Load in the Preprocessed Data
There are 400 frames for each camera in the dataset. Each frame exhibits several cropped boxes or rather pedestrians. For a single camera, 400 frames mean that there are 400 time steps available. In each time step we extract a snapshot of the underlying graph. Compared to the Chickenpox Dataset which has 517 snapshots, our dataset will only have 400 snapshots. 

In [3]:
with open('saved_dictionary-C1.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

We extract the re-id features and additional information from the loaded dictionary `loaded_dict`. In order to split the data into several snapshots, we need to determine the number of elements corresponding to the same `frame_ID`. The dictionary `snapshot_dict` is only one possibility to do so. 

In [4]:
re_id, person_ID, frame_ID, name, cam_ID = zip(*([loaded_dict[i][0], loaded_dict[i][1], 
                                                  loaded_dict[i][2], loaded_dict[i][3], 
                                                  loaded_dict[i][4]] for i in loaded_dict))
snapshot_dict = {int(i):frame_ID.count(i) for i in frame_ID}

### 2. Define the General Graph Topology 
It is crucial to know what information is actually needed to perform a specific task. Since we are going to prepare the temporal graph data for a link prediction task, here is one suggestion for how the graph components may be defined. 

| Graph Information | Equivalent Information | 
| :---   | :---     |
| Nodes | Persons |
| Edges | Connections |
| Node Features | ReID feature vectors of length 2048 |
| Targets (Node Labels) | Person ID |
| Edge Labels | 1 if the connected nodes represent the same person. Else 0.|
| Edge Weights | All-Ones vectors to specify unweighted graph. |
| Time Step | 1 Frame |
| Temporal Graph Type | Dynamic growing graph |

Note that our graph strictly grows and that every node will remain part of the graph. Further, in our case edge labels and edge features are the same. 

### 3. Create Edge list and Edge Labels
Now, we have to think about which nodes shall be connected. This information shall be stored in an edge list in COO format. My intuition was to connect every node with each other but this would result in a completed graph with obviously irrelevant connections. My second attempt was motivated by the desire to keep the number of connections as low as possible, thereby reducing the overall computation cost. Here is what I came up with:
- Observation 1: there won't be any connection between new nodes from the same time step, since a node represents a specific person and (s)he won't appear twice in the same frame (in the multi-camera MOT case this doesn't hold anymore!).
- Observation 2: if the term historic nodes refers to nodes from past snapshots we need to connect every historic node with every new node. 

The most relevant information for defining the edges and their labels is the `person_ID`. An ideal graph model would label the edges between nodes that have the same `person_ID` with a `1` and all others with `0`. Since in the real world we don't know the person's ID we cannot perform this node prediction and have to switch to the alternative link label prediction. The model might learn from the node features how to find similarities between nodes without knowing their actual person ID. 

We first determine the number of cropped boxes, i.e. the number of pedestrians, that are newly detected in each snapshot. Based on that we can determine the number of all nodes within each snapshot. The value `31` at position `1` in `num_cropped_boxes` signalizes that there are 31 new nodes in snapshot 1. The value `126` at position `3` in `num_nodes` means that there are 126 nodes in the graph snapshot at t = 3. 

In [5]:
num_cropped_boxes = []
for i in range(1,n_frames+1):
    num_cropped_boxes.append(snapshot_dict[i])
num_cropped_boxes 

[32, 31, 30, 33, 33, 33, 30, 30, 31, 31]

In [6]:
num_nodes = [num_cropped_boxes[0] if i == 1 else sum(num_cropped_boxes[0:i]) for i in range(1,n_frames+1)]
num_nodes 

[32, 63, 93, 126, 159, 192, 222, 252, 283, 314]

#### Create DataFrame Objects Containing the Mapping between Person's ID and Integer
The function `create_n_dataframes` creates as many pandas `DataFrame` objects as specified in the argument. Each dataframe corresponds to one snapshot and has as many rows as nodes are in the graph snapshot, respectively. We map the person's ID to a `node_ID` to emphasize the node order. Particularlym this step assigns integer values to the node labels and enables us to have different node ID's for the same node label if the same person appears in more than one time instance. 

In [7]:
def create_n_dataframes(n):
    for i in range(n):
        nodeID_personID_dict = {'node_ID': range(num_nodes[i]), 'person_ID': person_ID[:num_nodes[i]]}
        exec(f'''global df_{i} 
df_{i} = pd.DataFrame(nodeID_personID_dict)''')

In [8]:
create_n_dataframes(n_frames)

Calling this function creates as many dataframes as specified in `n_frames`. Note that you can access them easily by running the following code snippet.

In [9]:
df_9

,node_ID,person_ID
0,0,0000
1,1,0001
2,2,0002
3,3,0003
4,4,0004
...,...,...
309,309,0043
310,310,0044
311,311,0122
312,312,0151


#### Create DataFrame Objects Containing the Stacked Temporal Graph Data

This DataFrame shall contain the edges for all snapshots. The final output `df_edges` contains all edges in the final snapshot. For the extraction, we exploit observation 1 and observation 2 from above. The general procedure in each time step looks as follows: 
- determine all new nodes and the historic nodes 
- use the cartesian product to get all possible combinations between the historic nodes and new nodes 
- create a dataframe `df_new_edges` based on the cartesian product and 
- stack new edges (`df_new_edges`) on historic edges `df_stack` 

At t = 0, we don't have historic nodes and no connections between nodes shall be established. Remember: there is no edge between new nodes, and in this case we only have new nodes. The dataframe for t = 0 therefore is generated using the cartesian product of `df_0` with itself. We add a new column called `'edge'` and set the value for each of the node pairs to zero to indicate that there is no edge. 

Note: we re-assign variable df_stack in each iteration with the concatenated dataframe and do NOT append. Therefore, we are going to extract the edges for each snapshot using a fixed lower bound and a moving upper bound for indexing. 

In [10]:
def generate_temporal_dataframes(num_frames): 
    # dataframe based on cartesian product at t = 0; df_stack will be used as historic nodes
    df_stack = df_0.merge(df_0,how='cross')  
    
    # dataframe based on cartesian product plus additional column 'edge' at t = 0
    df_edges = df_stack.copy()
    df_edges['edge'] = False  
     
    for i in range(1,num_frames):
        # dataframe containing all new nodes in each time step
        exec(f'''global new_nodes
new_nodes = df_{i}[num_nodes[i-1]:num_nodes[i]+1]''')
        
        # dataframe based on cartesian product of new nodes and historic nodes in each time step t > 0
        exec(f'''global df_new_edges
df_new_edges = df_{i-1}.merge(new_nodes,how='cross')''')        
        
        # dataframe containing the new set of historic nodes for the next iteration
        df_stack = pd.concat([df_stack,df_new_edges], ignore_index=True)
        
        df_new_edges['edge'] = True
        df_edges = pd.concat([df_edges,df_new_edges])
    return df_stack,df_edges  

Call the function and check whether both returned dataframes contain the same column `'node_ID_x'`.

In [11]:
a,b = generate_temporal_dataframes(n_frames)
np.array_equal(a['node_ID_x'], b['node_ID_x'])

True

In [12]:
b

,node_ID_x,person_ID_x,node_ID_y,person_ID_y,edge
0,0,0000,0,0000,False
1,0,0000,1,0001,False
2,0,0000,2,0002,False
3,0,0000,3,0003,False
4,0,0000,4,0004,False
...,...,...,...,...,...
8768,282,0383,309,0043,True
8769,282,0383,310,0044,True
8770,282,0383,311,0122,True
8771,282,0383,312,0151,True


#### Create the Edge Labels
Recall that there are edges between all new nodes and the historic ones and that for t = 0 we don't have any connection. Note that all created connections (except for the snapshot at t=0) exist but they don't have the same labels. Since there is no edge between the nodes from t = 0 we set their labels to `0`. For all subsequent snapshots we label all existing edges (corresponds to the condition `b['edge']==True`) with a `1` if both nodes have the same person ID. In all other cases we set the label `0`, indicating that this edge connects two nodes that do not have the same labels.

In [13]:
b['labels'] = np.where((b['person_ID_x'] == b['person_ID_y']) & (b['edge']==True), 1, 0)
b

,node_ID_x,person_ID_x,node_ID_y,person_ID_y,edge,labels
0,0,0000,0,0000,False,0
1,0,0000,1,0001,False,0
2,0,0000,2,0002,False,0
3,0,0000,3,0003,False,0
4,0,0000,4,0004,False,0
...,...,...,...,...,...,...
8768,282,0383,309,0043,True,0
8769,282,0383,310,0044,True,0
8770,282,0383,311,0122,True,0
8771,282,0383,312,0151,True,0


In [14]:
b.to_excel("edge_connections.xlsx") 

#### Generate the Edge List and Edge Label List
Based on the dataframe b that contains sub-dataframes that correspond to each snapshot we create the edge list and edge label list. Both lists contain numpy arrays each of which corresponds to one snapshot. In the process, we introduce the edge weight matrix and initialize it with all ones for each snapshot to indicate that we are dealing with unweighted graphs in each snapshot.

In [15]:
edge_indices = []
edge_labels = []
edge_weights = []

In [16]:
source_nodes = b['node_ID_x'].to_list()
target_nodes = b['node_ID_y'].to_list()

Our main goal here is to extract all sub-dataframes. This was done by using the number of cropped boxes that was calculated in a previous code block.

In [17]:
num_cropped_boxes # number of new nodes for each time step

[32, 31, 30, 33, 33, 33, 30, 30, 31, 31]

Based on `num_cropped_boxes` we calculate upper bounds for the array indexing. The upper bound 1024 is the threshold for snapshot 0 at t = 0. The variable `upper` denotes the upper bound in each time step and is recursively calculated based on the current and previous number of new nodes. Since there are no connections for the nodes from snapshot t = 0 we can skip the first 32x32 = 1024 rows, where 32 is the number of new nodes for t = 0. Recall: the 1024 rows were created using the cartesian product. 

In [18]:
upper = num_cropped_boxes[0]**2 # 1024
for i in range(0,n_frames):
    upper = upper + num_cropped_boxes[i] * sum(num_cropped_boxes[:i]) # t=0: upper + 1024*0

    e_snap = np.transpose(np.array([list(a) for a in zip(source_nodes[1024:upper], target_nodes[1024:upper])]))
    edge_indices.append(e_snap)
    
    l_snap = np.array(b['labels'][1024:upper])
    edge_labels.append(l_snap)
    
    edge_weights.append(np.ones(l_snap.size))

Finally, we get the edge list in COO format.

In [19]:
edge_indices[0:2]

[array([], dtype=float64),
 array([[ 0,  0,  0, ..., 31, 31, 31],
        [32, 33, 34, ..., 60, 61, 62]])]

### 4. Extract Node Features and Node Labels
We start by recalling the number of total nodes in each snapshot as given in `num_nodes`.

In [20]:
features = []
targets = [] # node labels

In [21]:
num_nodes

[32, 63, 93, 126, 159, 192, 222, 252, 283, 314]

For all snapshots we use the corresponding number of nodes as upper bound to extract the node features and node labels. As defined at the beginning `re_id` is used as node features and `person_ID` as targets or rather node labels.

In [22]:
for i in range(n_frames):
    f_snap = re_id[:num_nodes[i]]
    features.append(f_snap)
    t_snap = np.array(person_ID[:num_nodes[i]])
    targets.append(t_snap) 

### 5. Generate the Temporal Graph Dataset

In [27]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
from torch_geometric_temporal.signal import temporal_signal_split

We choose the iterator `DynamicGraphTemporalSignal` and pass the computed parameters. This will create the temporal graph consisting of multiple snapshots of type `Data` (PyG object). In case that batches shall be used I'd suggest touse `Batch` objects instead of `Data` objects and the iterator `DynamicGraphTemporalSignalBatch`. Note that `edge_labels` is the only additional parameter. Also, I don't exactly know the difference between the used iterator and `DynamicGraphStaticSignal`. 

In [24]:
tg_dataset = DynamicGraphTemporalSignal(edge_indices=edge_indices,
                                            edge_weights=edge_weights,
                                            features=features,
                                            targets=targets,
                                            edge_labels=edge_labels)

Splits iterator in the temporal dimension using a pre-set ratio. This function divides the set of `Data` objects into two sets: the first 0.8% snapshots belong to the training set and the other 0.2% to the test set.

In [25]:
train_dataset, test_dataset = temporal_signal_split(tg_dataset, train_ratio=0.8)

In [28]:
for i in tg_dataset:
    print(i)

Data(x=[32, 2048], edge_index=[0], edge_attr=[0], edge_labels=[0])
Data(x=[63, 2048], edge_index=[2, 992], edge_attr=[992], edge_labels=[992])
Data(x=[93, 2048], edge_index=[2, 2882], edge_attr=[2882], edge_labels=[2882])
Data(x=[126, 2048], edge_index=[2, 5951], edge_attr=[5951], edge_labels=[5951])
Data(x=[159, 2048], edge_index=[2, 10109], edge_attr=[10109], edge_labels=[10109])
Data(x=[192, 2048], edge_index=[2, 15356], edge_attr=[15356], edge_labels=[15356])
Data(x=[222, 2048], edge_index=[2, 21116], edge_attr=[21116], edge_labels=[21116])
Data(x=[252, 2048], edge_index=[2, 27776], edge_attr=[27776], edge_labels=[27776])
Data(x=[283, 2048], edge_index=[2, 35588], edge_attr=[35588], edge_labels=[35588])
Data(x=[314, 2048], edge_index=[2, 44361], edge_attr=[44361], edge_labels=[44361])


The iterator `tg_dataset` is ready to be used for training a temporal GNN.